In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import scipy
import math
import os
import sys
print(f'python version: {sys.version}')
print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'nibabel version: {nib.__version__}')


NEAR_ZERO_THRESHOLD = 1e-6
def int2seg(intensity):
    if intensity <= 7:
        return f"C{intensity}"
    elif intensity >7 and intensity <= 19:
        return f"T{intensity-7}"
    else:
        return f"L{intensity-19}"
    
df = pd.read_csv("mendez_df.csv") #Mendez values
df.head()

python version: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:06) 
[Clang 14.0.6 ]
numpy version: 1.23.5
pandas version: 1.5.3
nibabel version: 5.1.0


,Segment,MEAN,STD,Mesure
0,C2,11.3,1.5,DREZ length
1,C2,12.6,0.9,Seg.L at dorsal CE
2,C2,13.1,1.0,Seg.L at BE
3,C2,24.4,0.8,Inf.Art facet to CR distance
4,C2,15.5,4.2,IF to RR distance


In [2]:
sct_dir = '../../spinalcordtoolbox/data/PAM50/template/'
print(sct_dir)

../../spinalcordtoolbox/data/PAM50/template/


In [3]:
# Import foramen data
#foramen_level = nib.load('PAM50_intervertebral_foramen_sandrine.nii.gz') # TODO add path
foramen_level = nib.load('test_foramen.nii.gz')

x_f,y_f,z_f = np.where(foramen_level.get_fdata()>NEAR_ZERO_THRESHOLD)
print(x_f, z_f)

[37 40 41 41 42 42 42 42] [932 731 691 898 765 797 830 865]


In [4]:
# Import Centerline data
center = nib.load(os.path.join(sct_dir, 'PAM50_centerline.nii.gz'))
x_c, y_c, z_c = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)
coord_ctl = np.where(center.get_fdata() > NEAR_ZERO_THRESHOLD)


# Import Cord data
PATH = os.path.join(sct_dir, 'PAM50_cord.nii.gz')
cord = nib.load(PATH)
p50_mask = cord.get_fdata()
x_cord, y_cord, z_cord = np.where(p50_mask>0)

## Estimate without STD
Take smallest distance bewtenn SC centerline (shifted by average diameter at dorsal rootlet entry and intervertebral foramen) (Sandrine)


In [5]:

len_level = {}
z_ref =  np.array(range(min(z_c), max(z_c) + 1))
z_c = z_ref
for i in range(len(x_f)):
    # Get coordinates of intervertebral foramen
    x = x_f[i]
    y = y_f[i]
    z = z_f[i]
    # Get level
    level = int(foramen_level.get_fdata()[x,y,z])
    print('Level', level)
    # Get correspondance of level number (2 --> C2, etc.)
    seg = int2seg(level)
    # Get mendez measures for the corresponding level
    seg_df = df[df.Segment == seg]
    RR = seg_df[seg_df.Mesure == "IF to RR distance"]
    CR = seg_df[seg_df.Mesure == "IF to CR distance"]
    W = seg_df.loc[seg_df.Mesure == "Dorsal width"].iloc[0]
    w_mean = W.MEAN # get mean value of dorsal width
    rr_mean = RR.MEAN.to_list()[0] # Get mean distance from rostral rootlet to interverteral foramen
    cr_mean = CR.MEAN.to_list()[0] # Get mean distance from caudal rootlet to interverteral foramen
    # Compute the 3D euclidean distance between intervertebral foramen and the back of the SC at dorsal width/2 lateral offset ( in mm)
    new_x = (x_c - w_mean)[0] # x value centerline - dorsal width /2 offset
    new_z = min(np.where(cord.get_fdata()[int(new_x),:,z] >0)[0]) # y value of the SC border at x = new_x and z = foramen
    distance_foramen_ctl = np.sqrt((x_c - w_mean - x)**2 + (new_z - y)**2  + (z_c - z)**2)*0.5  # 0.5 --> pix dim of PAM50 TODO: remove hardcode
    # Get the closest distance to the rostral rootlet compared to the Mendez value
    rostral_diff = np.array([np.abs(i - rr_mean) for i in distance_foramen_ctl])
    # Only use slices higher than the foramen
    rostral = np.argmin(rostral_diff[-len(z_c[z_c>z])-1::])
    z_ref_r = z_ref[-len(z_c[z_c>z])-1::]
    # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
    r_z = z_ref_r[rostral]

    # Get the closest distance to the caudal rootlet compared to the Mendez value
    caudal_diff = np.array([np.abs(i - cr_mean) for i in distance_foramen_ctl])
    # Only use slices higher than the foramen
    caudal = np.argmin(caudal_diff[-len(z_c[z_c>z])-1::])
    # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
    c_z = z_ref_r[caudal]

    print('Rostral', rr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][rostral], r_z, z)
    print('Caudal', cr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][caudal], c_z, z)
    # Set slices
    len_level[level] = (r_z, c_z)


Level 2
Rostral 15.5 15.459741265622785 949 932
Caudal 10.6 12.913310961949302 932 932
Level 8
Rostral 28.1 28.009641197273485 778 731
Caudal 17.0 16.986465200270477 746 731
Level 9
Rostral 33.4 33.19358710654815 751 691
Caudal 22.0 22.154327455375395 725 691
Level 3
Rostral 16.299999999999997 16.34864214545049 906 898
Caudal 10.8 15.851753846183708 898 898
Level 7
Rostral 20.1 20.195237062238217 792 765
Caudal 11.8 15.019906790656194 765 765
Level 6
Rostral 17.2 17.188952847686796 813 797
Caudal 10.3 15.213812802844656 797 797
Level 5
Rostral 15.1 15.168982332378135 830 830
Caudal 9.3 15.168982332378135 830 830
Level 4
Rostral 18.1 18.168599973580793 885 865
Caudal 10.1 15.168982332378135 865 865


In [6]:
cord_mask = np.copy(p50_mask)
for lvl in len_level:
    level_r = int(len_level[lvl][0])
    level_c = int(len_level[lvl][1])
    cord_mask[:, :, level_r][cord_mask[:, :, level_r]>0] = lvl
    cord_mask[:, :, level_c][cord_mask[:, :, level_c]>0] = lvl
cord_mask[:, :, :][cord_mask[:, :, :]==1] = 0
test_img = nib.Nifti1Image(cord_mask, header=cord.header, affine=cord.affine)
nib.save(test_img, f'foramen_label_without_std.nii.gz')  # TO CHANGE path output

## Estimate with STD 
Take smallest distance bewtenn SC centerline (shifted by average diameter at dorsal rootlet entry and intervertebral foramen) (Sandrine)


In [7]:
z_ref =  np.array(range(min(z_c), max(z_c) + 1))
z_c = z_ref
for i in range(len(x_f)):
    # Get coordinates of intervertebral foramen
    x = x_f[i]
    y = y_f[i]
    z = z_f[i]
    # Get level
    level = int(foramen_level.get_fdata()[x,y,z])
    print('Level', level)
    # Get correspondance of level number (2 --> C2, etc.)
    seg = int2seg(level)
    print(seg)
    # Get mendez measures for the corresponding level
    seg_df = df[df.Segment == seg]
    RR = seg_df[seg_df.Mesure == "IF to RR distance"]
    CR = seg_df[seg_df.Mesure == "IF to CR distance"]
    W = seg_df.loc[seg_df.Mesure == "Dorsal width"].iloc[0]
    w_mean = W.MEAN # get mean value of dorsal width
    rr_mean = RR.MEAN.to_list()[0] # Get mean distance from rostral rootlet to interverteral foramen
    cr_mean = CR.MEAN.to_list()[0] # Get mean distance from caudal rootlet to interverteral foramen
    rr_std = RR.STD.to_list()[0] # Get std for mean distance from rostral rootlet to interverteral foramen
    cr_std = CR.STD.to_list()[0] # Get std for mean distance from caudal rootlet to interverteral foramen
    # Get distance for +/- 95% probability
    min_rr = rr_mean - 2*rr_std
    max_rr = rr_mean + 2*rr_std
    min_cr = cr_mean - 2*cr_std
    max_cr = cr_mean + 2*cr_std
    print("Rostral dist", max_rr, rr_mean , min_rr)
    print("Caudal dist", max_cr, cr_mean , min_cr)
    # Compute the 3D euclidean distance between intervertebral foramen and the back of the SC at dorsal width/2 lateral offset ( in mm)
    new_x = (x_c - w_mean)[0] # x value centerline - dorsal width /2 offset
    new_z = min(np.where(cord.get_fdata()[int(new_x),:,z] >0)[0]) # y value of the SC border at x = new_x and z = foramen
    distance_foramen_ctl = np.sqrt((x_c - w_mean - x)**2 + (new_z - y)**2  + (z_c - z)**2)*0.5  # 0.5 --> pix dim of PAM50 TODO: remove hardcode
    # min std : most caudal point of the probabilistic distribution 
    # max std : most rostral point of the probabilistic distribution 
    # List with estimation of z value for rostral and caudal (min std, max std, mean)
    all_level = []
    for distance in [max_rr,min_rr,rr_mean,max_cr, min_cr, cr_mean]:
        # Get the closest distance to the rostral rootlet compared to the Mendez value
        rostral_diff = np.array([np.abs(i - distance) for i in distance_foramen_ctl])
        # Only use slices higher than the foramen
        estimate = np.argmin(rostral_diff[-len(z_c[z_c>z])-1::])
        z_ref_i = z_ref[-len(z_c[z_c>z])-1::]
        # Get the slice number (adjusted since the centerline starts at slice 55 not 0)
        value = z_ref_i[estimate]
        all_level.append(int(value))

    print('Rostral', rr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][rostral], all_level[2] , z)
    print('Caudal', cr_mean, distance_foramen_ctl[-len(z_c[z_c>z])-1::][caudal],all_level[5] , z)
    # Set colored label from 0 (the least probable) to 20 (mean value)
    cord_mask = np.copy(p50_mask)
    # Max to mean rostral
    for rng in range(all_level[2], all_level[0]):
        if abs(all_level[0]-all_level[2]) !=0:
            proba = 19*abs(rng-all_level[2])/(abs(all_level[0]-all_level[2])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = 20-proba
    # Mean to min rostral 
    for rng in range(all_level[1], all_level[2]):
        if abs(all_level[2]-all_level[1]) != 0:
            proba = 19*abs(rng-all_level[1])/(abs(all_level[2]-all_level[1])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = proba
    # Max to mean caudal
    for rng in range(all_level[5], all_level[3]):
        if abs(all_level[3]-all_level[5]) !=0:
            proba = 19*abs(rng-all_level[5])/(abs(all_level[3]-all_level[5])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = 20-proba
    # Mean to min caudal
    for rng in range(all_level[4], all_level[5]):
        if abs(all_level[5]-all_level[4]) != 0:
            proba = 19*abs(rng-all_level[4])/(abs(all_level[5]-all_level[4])) +1
            cord_mask[:, :, rng][cord_mask[:, :, rng]>0] = proba
    # Mean value caudal and rostral = 20
    for l in (all_level[2], all_level[5]):
        cord_mask[:, :, l][cord_mask[:, :, l]>0] = 20
    cord_mask[:, :, :][cord_mask[:, :, :]==1] = 0
    test_img = nib.Nifti1Image(cord_mask, header=cord.header, affine=cord.affine)
    nib.save(test_img, f'Outputs/spinal_level_std_{level}.nii.gz')  # TO CHANGE path output

Level 2
C2
Rostral dist 23.9 15.5 7.1
Caudal dist 12.0 10.6 9.2
Rostral 15.5 16.33259318050872 949 932
Caudal 10.6 12.913310961949302 932 932
Level 8
T1
Rostral dist 35.9 28.1 20.3
Caudal dist 22.8 17.0 11.2
Rostral 28.1 18.228823330100052 778 731
Caudal 17.0 15.241062955056643 746 731
Level 9
T2
Rostral dist 44.6 33.4 22.199999999999996
Caudal dist 29.6 22.0 14.4
Rostral 33.4 17.372801299732867 751 691
Caudal 22.0 14.206133358518072 725 691
Level 3
C3
Rostral dist 21.299999999999997 16.299999999999997 11.299999999999997
Caudal dist 16.200000000000003 10.8 5.4
Rostral 16.299999999999997 18.74241446559114 906 898
Caudal 10.8 15.851753846183708 898 898
Level 7
C7
Rostral dist 21.700000000000003 20.1 18.5
Caudal dist 13.600000000000001 11.8 10.0
Rostral 20.1 18.044323207036612 792 765
Caudal 11.8 15.019906790656194 765 765
Level 6
C6
Rostral dist 19.599999999999998 17.2 14.799999999999999
Caudal dist 12.3 10.3 8.3
Rostral 17.2 18.20604569916268 813 797
Caudal 10.3 15.213812802844656 797 7